In [2]:
import torch
import torchani
from torchani.datasets import ANIDataset
from torchani.units import hartree2kcalmol
from pathlib import Path
import ase
from ase.visualize import view

In [3]:
device = torch.device('cpu')
ani2x = torchani.models.ANI2x().to(device).double()
ani1ccx = torchani.models.ANI1ccx().to(device).double()
ds_path = Path('/home/nick/First_DSs/ANI-1x-first-conformers.h5')
ds = ANIDataset(locations=ds_path, names='1x-first')



ds_forces = hartree2kcalmol(ds['CH4']['forces'].to(device))
print(ds_forces)

tensor([[[-10.1421,   1.7922, 126.5073],
         [-14.6820, -17.4242, -21.5316],
         [  8.2877, -22.1501, -24.9454],
         [ 23.2253,  45.4318, -71.4660],
         [ -6.6889,  -7.6498,  -8.5643]]], dtype=torch.float64)


In [4]:
torch.linalg.norm(ds_forces, dim=-1)

tensor([[126.9258,  31.3492,  34.3742,  87.8114,  13.2894]],
       dtype=torch.float64)

In [5]:
sq_ds_forces = ds_forces.square()
sqrt_sq_sum_forces = (sq_ds_forces.sum(dim=-1).sqrt()).unsqueeze(-1)
normalized_forces =ds_forces / sqrt_sq_sum_forces

In [6]:
normalized_forces

tensor([[[-0.0799,  0.0141,  0.9967],
         [-0.4683, -0.5558, -0.6868],
         [ 0.2411, -0.6444, -0.7257],
         [ 0.2645,  0.5174, -0.8139],
         [-0.5033, -0.5756, -0.6444]]], dtype=torch.float64)

In [7]:
ch4_species = ds['CH4']['species'].to(device)
ch4_coord = ds['CH4']['coordinates'].to(device)
inp = (ch4_species, ch4_coord)
members_forces = hartree2kcalmol(ani2x.members_forces(inp).model_forces)
print(members_forces.std(0))

tensor([[0.2954, 0.7690, 0.5616],
        [0.2051, 0.1653, 0.1885],
        [0.1333, 0.2251, 0.3848],
        [0.3154, 0.4854, 0.4680],
        [0.1293, 0.1516, 0.1282]])


In [8]:
members_forces[0]

tensor([[-10.1069,   1.8989, 126.1252],
        [-14.3177, -17.3228, -21.2256],
        [  8.1441, -21.9234, -25.4874],
        [ 22.9873,  45.3785, -70.7184],
        [ -6.7068,  -8.0312,  -8.6938]])

In [9]:
ccx_forces = hartree2kcalmol(ani1ccx.members_forces(inp).model_forces)
#print(ccx_forces)
#print(torch.linalg.norm(ccx_forces, dim=-1).std(0))

In [10]:
norm_ds_forces = torch.nn.functional.normalize(ds_forces, dim=-1)
norm_ani_forces = torch.nn.functional.normalize(members_forces, dim=-1)
print(norm_ani_forces[0].shape, norm_ds_forces.squeeze(0).shape)

torch.Size([5, 3]) torch.Size([5, 3])


In [11]:
torch.linalg.norm(ds_forces, dim=-1)

tensor([[126.9258,  31.3492,  34.3742,  87.8114,  13.2894]],
       dtype=torch.float64)

In [12]:
torch.linalg.norm(members_forces.mean(0), dim=-1)

tensor([126.5278,  31.1284,  34.1294,  87.1091,  13.6333])

In [13]:
mag_members_forces = torch.linalg.norm(members_forces, dim=-1)
#print(mag_members_forces)
mag_ds_forces = torch.linalg.norm(ds_forces, dim=-1)
#print(mag_ds_forces)
print(mag_members_forces / mag_ds_forces)
print((mag_members_forces/mag_ds_forces).std(0))
print((mag_members_forces/mag_ds_forces).std(-1))

tensor([[0.9970, 0.9861, 1.0063, 0.9920, 1.0237],
        [0.9910, 0.9940, 0.9667, 0.9994, 1.0188],
        [1.0004, 0.9967, 0.9984, 0.9906, 1.0347],
        [1.0002, 0.9871, 0.9868, 1.0007, 1.0252],
        [0.9929, 0.9929, 1.0044, 0.9787, 1.0228],
        [0.9978, 1.0034, 0.9933, 0.9848, 1.0426],
        [1.0035, 1.0012, 0.9976, 0.9985, 1.0283],
        [0.9922, 0.9824, 0.9897, 0.9914, 1.0118]], dtype=torch.float64)
tensor([0.0045, 0.0074, 0.0125, 0.0076, 0.0095], dtype=torch.float64)
tensor([0.0147, 0.0187, 0.0174, 0.0156, 0.0164, 0.0224, 0.0128, 0.0109],
       dtype=torch.float64)


In [14]:
members_forces.std(0)

tensor([[0.2954, 0.7690, 0.5616],
        [0.2051, 0.1653, 0.1885],
        [0.1333, 0.2251, 0.3848],
        [0.3154, 0.4854, 0.4680],
        [0.1293, 0.1516, 0.1282]])

In [15]:
torch.linalg.norm(members_forces.std(0), dim=-1)

tensor([0.9971, 0.3240, 0.4653, 0.7444, 0.2370])

In [16]:
mag_members_forces.std(0)

tensor([0.5674, 0.2328, 0.4309, 0.6682, 0.1256])

In [17]:
cos = torch.nn.CosineSimilarity(dim=-1)

In [18]:
cos(ds_forces, members_forces.mean(0))

tensor([[1.0000, 1.0000, 0.9999, 1.0000, 0.9999]], dtype=torch.float64)

In [27]:
import pandas as pd
import math

df = pd.DataFrame(cos(norm_ds_forces, norm_ani_forces))
df.index+=1
df = df.rename(columns={0: "C", 1: "H", 2: "H", 3: "H", 4: "H"})
df.round(4)

,C,H,H,H,H
1,1.0000,1.0000,0.9999,1.0,0.9998
2,1.0000,1.0000,1.0000,1.0,0.9994
3,1.0000,0.9999,1.0000,1.0,1.0000
4,1.0000,0.9998,0.9999,1.0,1.0000
5,1.0000,1.0000,0.9998,1.0,0.9998
6,0.9999,1.0000,0.9999,1.0,0.9995
7,1.0000,1.0000,0.9999,1.0,0.9998
8,1.0000,1.0000,0.9999,1.0,1.0000


In [20]:
torch.set_printoptions(precision=4)
cos(ds_forces, members_forces)

tensor([[1.0000, 1.0000, 0.9999, 1.0000, 0.9998],
        [1.0000, 1.0000, 1.0000, 1.0000, 0.9994],
        [1.0000, 0.9999, 1.0000, 1.0000, 1.0000],
        [1.0000, 0.9998, 0.9999, 1.0000, 1.0000],
        [1.0000, 1.0000, 0.9998, 1.0000, 0.9998],
        [0.9999, 1.0000, 0.9999, 1.0000, 0.9995],
        [1.0000, 1.0000, 0.9999, 1.0000, 0.9998],
        [1.0000, 1.0000, 0.9999, 1.0000, 1.0000]], dtype=torch.float64)

In [21]:
print(members_forces.mean(0))
print(ds_forces)
print(torch.sub(members_forces.mean(0), ds_forces))
diff = torch.sub(members_forces.mean(0), ds_forces)
print(torch.linalg.norm(diff, dim=-1))
#print(torch.sub(members_forces, ds_forces))

tensor([[ -9.8007,   1.9067, 126.1332],
        [-14.4167, -17.3474, -21.4524],
        [  8.0677, -21.7298, -25.0508],
        [ 22.8722,  45.1770, -70.8795],
        [ -6.7225,  -8.0066,  -8.7505]])
tensor([[[-10.1421,   1.7922, 126.5073],
         [-14.6820, -17.4242, -21.5316],
         [  8.2877, -22.1501, -24.9454],
         [ 23.2253,  45.4318, -71.4660],
         [ -6.6889,  -7.6498,  -8.5643]]], dtype=torch.float64)
tensor([[[ 0.3414,  0.1145, -0.3740],
         [ 0.2652,  0.0768,  0.0792],
         [-0.2200,  0.4203, -0.1055],
         [-0.3530, -0.2548,  0.5865],
         [-0.0336, -0.3567, -0.1862]]], dtype=torch.float64)
tensor([[0.5192, 0.2873, 0.4860, 0.7304, 0.4038]], dtype=torch.float64)


In [22]:
difference = torch.sub(members_forces, ds_forces)
print(torch.linalg.norm(difference, dim=-1).std(0))
print(difference.mean(0))

tensor([0.3804, 0.1982, 0.2664, 0.5895, 0.1488], dtype=torch.float64)
tensor([[ 0.3414,  0.1145, -0.3741],
        [ 0.2652,  0.0768,  0.0792],
        [-0.2200,  0.4203, -0.1055],
        [-0.3530, -0.2548,  0.5865],
        [-0.0336, -0.3567, -0.1862]], dtype=torch.float64)


In [23]:
(members_forces / difference).std(0)

tensor([[ 186.5916,   13.5331, 1612.2441],
        [ 941.3034,  696.9122,  437.6643],
        [  36.1232,   36.7143,  450.3755],
        [ 340.2724, 7912.5770,  277.1345],
        [ 289.8580,   12.4289,   75.1645]], dtype=torch.float64)

In [24]:
mean_forces = members_forces.mean(0)
difference_mean = difference.mean(0)
stdev_forces = members_forces.std(0)
coeff_var = stdev_forces / mean_forces
diff_var = stdev_forces / difference_mean
print(diff_var)

tensor([[ 0.8654,  6.7186, -1.5015],
        [ 0.7735,  2.1513,  2.3789],
        [-0.6060,  0.5356, -3.6488],
        [-0.8935, -1.9050,  0.7980],
        [-3.8524, -0.4250, -0.6886]], dtype=torch.float64)


In [25]:
coeff_var

tensor([[-0.0301,  0.4033,  0.0045],
        [-0.0142, -0.0095, -0.0088],
        [ 0.0165, -0.0104, -0.0154],
        [ 0.0138,  0.0107, -0.0066],
        [-0.0192, -0.0189, -0.0147]])

In [26]:
ch4_coord

tensor([[[ 4.9725e-04, -2.3656e-02, -4.6554e-02],
         [-9.4934e-01, -4.6713e-01, -2.1225e-01],
         [-2.1828e-01,  6.4611e-01,  8.7319e-01],
         [ 3.7291e-01,  6.5190e-01, -6.9571e-01],
         [ 7.9173e-01, -6.8895e-01,  3.1410e-01]]], requires_grad=True)

In [46]:
from torchani.utils import PERIODIC_TABLE
from ase.optimize import BFGS

def tensor_from_xyz(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        num_atoms = int(lines[0])
        coordinates = []
        species = []
        _, a, b, c = lines[2].split()
        cell = torch.diag(torch.tensor([float(a), float(b), float(c)]))
        for line in lines[2:]:
            values = line.split()
            if values:
                s = values[0].strip()
                x = float(values[1])
                y = float(values[2])
                z = float(values[3])
                coordinates.append([x, y, z])
                species.append(PERIODIC_TABLE.index(s))
        coordinates = torch.tensor(coordinates)
        species = torch.tensor(species, dtype=torch.long)
        assert coordinates.shape[0] == num_atoms
        assert species.shape[0] == num_atoms
    return species, coordinates, cell

In [28]:
calculator = ani2x.ase()
ch5_species = tensor_from_xyz('ch5.xyz')[0][None].to(device)
ch5_coord = tensor_from_xyz('ch5.xyz')[1][None].to(device)
ch5_inp = (ch5_species, ch5_coord)
ani2x(ch5_inp)

SpeciesEnergies(species=tensor([[1, 0, 0, 0, 0, 0]]), energies=tensor([-41.0184], dtype=torch.float64))

In [29]:
ase_ch5 = ase.Atoms(numbers=ch5_species[0],positions=ch5_coord[0])
ase_ch5.calc = calculator

dyn = BFGS(ase_ch5)
dyn.run(fmax=0.05)

ch5_coord = torch.tensor(ase_ch5.positions[None])
ch5 = (ch5_species, ch5_coord)

print(ch5)

      Step     Time          Energy         fmax
BFGS:    0 13:30:49    -1116.167198        0.4223
BFGS:    1 13:30:49    -1116.175872        0.2969
BFGS:    2 13:30:49    -1116.182091        0.0541
BFGS:    3 13:30:49    -1116.182234        0.0311
(tensor([[6, 1, 1, 1, 1, 1]]), tensor([[[ 1.5394e+00, -3.3886e-01,  4.9230e-06],
         [ 2.4156e+00, -1.0414e+00,  1.0968e-01],
         [ 6.6337e-01,  3.6378e-01, -1.0981e-01],
         [ 1.1664e+00, -6.5049e-01,  9.8277e-01],
         [ 1.2221e+00, -8.7541e-01, -9.0197e-01],
         [ 2.2296e+00,  5.0922e-01, -8.0671e-02]]], dtype=torch.float64))


In [30]:
view(ase_ch5, viewer='ngl')

In [31]:
torch.set_printoptions(sci_mode=False)

In [32]:
ch5_forces = hartree2kcalmol(ani2x.members_forces(ch5_inp).model_forces)
print(ch5_forces)

tensor([[[    -0.0003,     -0.0794,      0.0568],
         [    37.3512,    -29.9500,      4.6758],
         [   -37.3827,     29.9815,     -4.6844],
         [    -1.9320,     -1.6045,      5.1067],
         [    -1.6343,     -2.7705,     -4.7140],
         [     3.5981,      4.4229,     -0.4408]],

        [[     0.0117,     -0.0476,      0.0288],
         [     9.8832,     -7.9304,      1.2405],
         [    -9.9174,      7.9483,     -1.2382],
         [     4.4730,      3.7434,    -11.7686],
         [     3.8161,      6.4443,     10.7878],
         [    -8.2666,    -10.1581,      0.9496]],

        [[     0.0279,     -0.0579,      0.0288],
         [    -9.4915,      7.6063,     -1.1860],
         [     9.4421,     -7.5773,      1.1872],
         [     7.5727,      6.3327,    -19.9351],
         [     6.4527,     10.9035,     18.2846],
         [   -14.0038,    -17.2074,      1.6205]],

        [[     0.0094,     -0.0606,      0.0391],
         [    10.7720,     -8.6459,      1.3

In [36]:
h_force = []
for i in ch5_forces:
    #print(i[1])
    print(torch.linalg.norm(i[1]))
    h_force.append(i[1])
print(h_force)

tensor(48.1038)
tensor(12.7321)
tensor(12.2209)
tensor(13.8788)
tensor(11.8998)
tensor(15.1998)
tensor(11.5172)
tensor(0.2167)
[tensor([ 37.3512, -29.9500,   4.6758]), tensor([ 9.8832, -7.9304,  1.2405]), tensor([-9.4915,  7.6063, -1.1860]), tensor([10.7720, -8.6459,  1.3538]), tensor([-9.2429,  7.4055, -1.1541]), tensor([11.7998, -9.4661,  1.4796]), tensor([ 8.9427, -7.1709,  1.1191]), tensor([ 0.1709, -0.1320,  0.0170])]


In [34]:
import numpy as np
np.cov(h_force)

/home/nick/miniconda3/envs/cuaev/lib/python3.9/site-packages/numpy/lib/function_base.py:2453: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  m = np.asarray(m)
/home/nick/miniconda3/envs/cuaev/lib/python3.9/site-packages/numpy/lib/function_base.py:2453: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  m = np.asarray(m)


AttributeError: 'float' object has no attribute 'shape'

In [58]:
_, __, ch5_mean_force, ch5_force_std = ani2x.force_qbcs(ch5_inp)
print(hartree2kcalmol(ch5_mean_force), '\n', hartree2kcalmol(ch5_force_std))

tensor([[ 0.01985, -0.05459,  0.03012],
        [ 7.52317, -6.03545,  0.94320],
        [-7.56261,  6.06227, -0.94492],
        [ 2.81397,  2.35756, -7.39845],
        [ 2.40496,  4.05892,  6.77859],
        [-5.19934, -6.38872,  0.59146]]) 
 tensor([[    0.01124,     0.01230,     0.01283],
        [   14.88431,    11.93420,     1.86310],
        [   14.87956,    11.93442,     1.86539],
        [    3.02032,     2.52252,     7.95375],
        [    2.57108,     4.34531,     7.30170],
        [    5.58916,     6.86913,     0.65204]])


In [59]:
ch5_mean_force = hartree2kcalmol(ch5_mean_force)
print(ch5_mean_force)

tensor([[ 0.01985, -0.05459,  0.03012],
        [ 7.52317, -6.03545,  0.94320],
        [-7.56261,  6.06227, -0.94492],
        [ 2.81397,  2.35756, -7.39845],
        [ 2.40496,  4.05892,  6.77859],
        [-5.19934, -6.38872,  0.59146]])


In [60]:
ch5_force_std = hartree2kcalmol(ch5_force_std)

In [61]:
ch5_mean_force

tensor([[ 0.01985, -0.05459,  0.03012],
        [ 7.52317, -6.03545,  0.94320],
        [-7.56261,  6.06227, -0.94492],
        [ 2.81397,  2.35756, -7.39845],
        [ 2.40496,  4.05892,  6.77859],
        [-5.19934, -6.38872,  0.59146]])

In [62]:
ch5_force_std

tensor([[    0.01124,     0.01230,     0.01283],
        [   14.88431,    11.93420,     1.86310],
        [   14.87956,    11.93442,     1.86539],
        [    3.02032,     2.52252,     7.95375],
        [    2.57108,     4.34531,     7.30170],
        [    5.58916,     6.86913,     0.65204]])

In [63]:
torch.set_printoptions(precision=5)
cos(ch5_mean_force, ch5_forces)

tensor([[ 0.94558,  1.00000,  1.00000, -0.99999, -0.99996, -0.99998],
        [ 0.99420,  1.00000,  1.00000,  1.00000,  1.00000,  1.00000],
        [ 0.99425, -1.00000, -1.00000,  1.00000,  1.00000,  1.00000],
        [ 0.98178,  1.00000,  1.00000,  1.00000,  1.00000,  1.00000],
        [ 0.96624, -1.00000, -1.00000,  1.00000,  1.00000,  1.00000],
        [ 0.96766,  1.00000,  1.00000,  1.00000,  1.00000,  1.00000],
        [ 0.98836,  1.00000,  1.00000,  1.00000,  0.99998,  0.99999],
        [ 0.99413,  0.99965,  0.99984,  0.99999,  0.99997,  0.99999]])

In [47]:
calculator = ani2x.ase()
epoxide_species = tensor_from_xyz('m-epoxide.xyz')[0][None].to(device)
epoxide_coord = tensor_from_xyz('m-epoxide.xyz')[1][None].to(device)
epoxide_inp = (epoxide_species, epoxide_coord)
ani2x(epoxide_inp)

SpeciesEnergies(species=tensor([[3, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), energies=tensor([-309.4919], dtype=torch.float64))

In [50]:
ase_epoxide = ase.Atoms(numbers=epoxide_species[0],positions=epoxide_coord[0])
ase_epoxide.calc = calculator

dyn = BFGS(ase_epoxide)
dyn.run(fmax=0.05)

epoxide_coord = torch.tensor(ase_epoxide.positions[None])
epoxide = (epoxide_species, epoxide_coord)

print(epoxide)

      Step     Time          Energy         fmax
BFGS:    0 09:35:32    -8429.407349        0.0437
(tensor([[8, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), tensor([[[ 0.7408,  0.8655, -2.6953],
         [ 2.0209,  1.3860, -2.3486],
         [ 2.1460,  1.7916, -0.8962],
         [ 1.7966, -0.0546, -2.4252],
         [ 1.7455, -0.6280, -1.0252],
         [ 1.5580,  0.6005, -0.0954],
         [ 0.0966,  0.8305,  0.2897],
         [ 3.2070,  1.9396, -0.6618],
         [ 1.6269,  2.7351, -0.6882],
         [ 2.6903, -1.1433, -0.8156],
         [ 0.9370, -1.3614, -0.9157],
         [ 2.5146,  1.9625, -3.1296],
         [ 2.1111, -0.6682, -3.2687],
         [-0.3008, -0.0431,  0.8170],
         [-0.5280,  1.0137, -0.5871],
         [ 0.0117,  1.6969,  0.9544],
         [ 2.1237,  0.4547,  0.8335]]], dtype=torch.float64))


In [52]:
epoxide_forces = hartree2kcalmol(ani2x.members_forces(epoxide_inp).model_forces)
_, __, epoxide_mean_force, epoxide_force_std = ani2x.force_qbcs(epoxide_inp)
#magnitudes = 
#mean_magnitudes = 
print(hartree2kcalmol(epoxide_mean_force), '\n', hartree2kcalmol(epoxide_force_std))
epoxide_mean_force = hartree2kcalmol(epoxide_mean_force)
epoxide_force_std = hartree2kcalmol(epoxide_force_std)
cos(epoxide_mean_force, epoxide_forces)

tensor([[-252.9948,   46.6740, -146.2518],
        [ 249.7320,  187.2523,  146.8823],
        [ -60.1982,    2.6833,   38.5559],
        [  83.5770, -242.8198,   75.1281],
        [ -12.0225,   19.4146,   49.7183],
        [ -10.2523,   -3.5441,   -4.4934],
        [  21.5389,    0.3345,  -12.1190],
        [ -21.5709,    3.0174,   -9.6478],
        [  12.5337,   -1.4670,   -8.0267],
        [ -18.5703,    0.5075,    0.7169],
        [   0.7064,   19.9252,  -15.9217],
        [  21.8640,   12.7613,  -57.2957],
        [ -29.0026,  -36.4934,  -61.0527],
        [  -8.0826,    1.9702,   10.4433],
        [   6.0939,   -3.6634,  -16.3681],
        [  -3.4760,   -3.6775,   12.0514],
        [  20.1243,   -2.8753,   -2.3193]]) 
 tensor([[21.3139,  8.4321, 34.3789],
        [15.8713, 24.1616, 12.6874],
        [ 9.1037,  7.7733,  9.1381],
        [16.2687, 25.3591, 18.2675],
        [ 6.0134,  9.8199,  7.0351],
        [ 2.2864,  3.0138, 11.2335],
        [ 2.7265,  1.3888,  4.6482],
       

tensor([[0.9926, 0.9985, 0.9968, 0.9971, 0.9869, 0.9218, 0.9961, 0.9977, 0.9979,
         0.9847, 0.9904, 0.9942, 0.9926, 0.9999, 0.9994, 1.0000, 0.9999],
        [0.9992, 0.9968, 0.9976, 0.9945, 0.9708, 0.7521, 0.9992, 0.9956, 0.9830,
         0.9768, 0.9996, 0.9985, 0.9990, 0.9999, 0.9904, 0.9998, 0.9978],
        [0.9906, 0.9977, 0.9954, 0.9981, 0.9960, 0.9819, 0.9977, 0.9990, 0.9991,
         0.9936, 0.9944, 0.9853, 0.9894, 0.9991, 0.9957, 0.9986, 0.9939],
        [0.9971, 0.9978, 0.9478, 0.9972, 0.9783, 0.7734, 0.9962, 0.9846, 0.9770,
         0.9331, 0.9800, 0.9994, 0.9990, 0.9995, 0.9984, 0.9989, 0.9998],
        [0.9985, 0.9989, 0.9992, 0.9978, 0.9955, 0.4425, 0.9978, 0.9970, 0.9548,
         0.9947, 0.9977, 0.9926, 0.9908, 1.0000, 0.9997, 0.9998, 0.9997],
        [0.9999, 0.9994, 0.9958, 0.9972, 0.9875, 0.9071, 0.9988, 0.9999, 0.9939,
         0.9967, 0.9984, 0.9999, 0.9956, 0.9995, 0.9979, 0.9994, 0.9986],
        [1.0000, 0.9972, 0.9890, 0.9966, 0.9677, 0.7240, 0.9905, 0.999

In [55]:
print(epoxide[0])

tensor([[8, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [60]:
import pandas as pd

df = pd.DataFrame(cos(epoxide_mean_force, epoxide_forces))
df.index+=1
df = df.rename(columns={0:'O', 1:'C', 2:'C', 3:'C', 4:'C', 5:'C', 6:'C', 7:'H', 8:'H', 9:'H', 10:'H', 11:'H', 12:'H', 13:'H', 14:'H', 15:'H', 16:'H'})
df.round(3)

,O,C,C,C,C,C,C,H,H,H,H,H,H,H,H,H,H
1,0.993,0.999,0.997,0.997,0.987,0.922,0.996,0.998,0.998,0.985,0.990,0.994,0.993,1.000,0.999,1.000,1.000
2,0.999,0.997,0.998,0.995,0.971,0.752,0.999,0.996,0.983,0.977,1.000,0.998,0.999,1.000,0.990,1.000,0.998
3,0.991,0.998,0.995,0.998,0.996,0.982,0.998,0.999,0.999,0.994,0.994,0.985,0.989,0.999,0.996,0.999,0.994
4,0.997,0.998,0.948,0.997,0.978,0.773,0.996,0.985,0.977,0.933,0.980,0.999,0.999,0.999,0.998,0.999,1.000
5,0.998,0.999,0.999,0.998,0.995,0.442,0.998,0.997,0.955,0.995,0.998,0.993,0.991,1.000,1.000,1.000,1.000
6,1.000,0.999,0.996,0.997,0.988,0.907,0.999,1.000,0.994,0.997,0.998,1.000,0.996,1.000,0.998,0.999,0.999
7,1.000,0.997,0.989,0.997,0.968,0.724,0.991,0.999,0.998,0.997,0.999,0.996,0.998,0.998,0.990,0.998,0.997
8,0.997,0.998,0.981,0.992,0.956,0.484,0.926,0.996,0.977,0.936,0.993,0.986,0.974,0.999,0.994,0.999,1.000
